<a href="https://colab.research.google.com/github/RaoEhsanElahi/Parallel_Processing/blob/main/MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y mpich
!pip install mpi4py


In [18]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_skxmte6
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_skxmte6
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done


## Task 1: Scatter and gathera 3x3 Matrix
####1.Generate a random 3x3 matrix on the master process.
####2.Scatter the rows of the matrix among MPI processes.
####3.Each process receives a subset of rows.
####4.Gather the results back to the master process.
####5.Verify that the gathered matrix is the same as the original matrix.

In [31]:
#%%writefile mpi-task1.py
from mpi4py import MPI
import numpy as np

MATRIX_SIZE = 3

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

    # Master-process: generates a random matrix
if rank == 0:
    original_matrix = np.random.rand(MATRIX_SIZE, MATRIX_SIZE)
    print("Original matrix (rank 0):")
    print(original_matrix)
else:
    original_matrix = None

    # Scatter rows to all processes
local_rows = np.empty((MATRIX_SIZE // size, MATRIX_SIZE), dtype=np.float64)
comm.Scatter(original_matrix, local_rows, root=0)

print(f"Local rows on rank {rank}:")
print(local_rows)

    # Gather rows back to the master-process
gathered_matrix = None
if rank == 0:
    gathered_matrix = np.empty((MATRIX_SIZE, MATRIX_SIZE), dtype=np.float64)

comm.Gather(local_rows, gathered_matrix, root=0)

    # Print verification results.
if rank == 0:
    match = np.array_equal(original_matrix, gathered_matrix)
    print("Gathered matrix (rank 0):")
    print(gathered_matrix)

    if match:
        print("Gathered matrix == original matrix.")
    else:
        print("Gathered matrix != original matrix.")


Original matrix (rank 0):
[[0.71720376 0.14841768 0.62912215]
 [0.04950821 0.59649635 0.50013654]
 [0.03968691 0.68583757 0.83165119]]
Local rows on rank 0:
[[0.71720376 0.14841768 0.62912215]
 [0.04950821 0.59649635 0.50013654]
 [0.03968691 0.68583757 0.83165119]]
Gathered matrix (rank 0):
[[0.71720376 0.14841768 0.62912215]
 [0.04950821 0.59649635 0.50013654]
 [0.03968691 0.68583757 0.83165119]]
Gathered matrix == original matrix.


In [30]:
!mpirun --allow-run-as-root -np 4 python mpi-task1.py

##Task 2: Matrix Addition with Scatter and Gather
####1.Generate two random matrices, A and B, where the dimensions (nxm) are user-defined.
####2.Scatter the rows of A and B among MPI processes.
####3.Each process adds its assigned rows of A and B.
####4.Gather the results back to the master process.
####5.Verify that the gathered matrix is the sum of A and B.

In [33]:
#%%writefile mpi-task2.py
from mpi4py import MPI
import numpy as np

def generate_random_matrix(rows, cols):
    return np.random.rand(rows, cols)

def matrix_addition(A, B):
    return A + B

# MPI setup
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

rows = 3
cols = 3

# Master process generates two random matrices A and B
if rank == 0:
    A = generate_random_matrix(rows, cols)
    B = generate_random_matrix(rows, cols)
    print("Matrix A (rank 0):")
    print(A)
    print("Matrix B (rank 0):")
    print(B)
else:
    A = None
    B = None

# Scatter process:
local_rows_A = np.empty((rows // size, cols), dtype=np.float64)
local_rows_B = np.empty((rows // size, cols), dtype=np.float64)
comm.Scatter(A, local_rows_A, root=0)
comm.Scatter(B, local_rows_B, root=0)

local_result = matrix_addition(local_rows_A, local_rows_B)

# Gather process:
gathered_result = None
if rank == 0:
    gathered_result = np.empty((rows, cols), dtype=np.float64)

comm.Gather(local_result, gathered_result, root=0)

# Print results:
if rank == 0:
    expected_result = matrix_addition(A, B)
    match = np.array_equal(gathered_result, expected_result)

    print("C: ")
    print(gathered_result)

    if match:
        print("C: the gathered matrix is the sum of Matrix A and Matrix B.")
    else:
        print("The gathered matrix 'C' is not the sum.")


Matrix A (rank 0):
[[0.40546317 0.02083776 0.33028951]
 [0.39360675 0.04463186 0.69018745]
 [0.93866435 0.01013458 0.14578521]]
Matrix B (rank 0):
[[0.83850059 0.82940833 0.30096155]
 [0.98590015 0.24860831 0.21806105]
 [0.70664615 0.82210424 0.42684646]]
C: 
[[1.24396376 0.85024608 0.63125106]
 [1.37950691 0.29324017 0.9082485 ]
 [1.6453105  0.83223882 0.57263167]]
C: the gathered matrix is the sum of Matrix A and Matrix B.


In [32]:
!mpirun --allow-run-as-root -np 4 python mpi_example.cpp    #un-check first line code to run this part.